In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

## Using Cartopy to Plot Maps

To create mapped output we need another module, Cartopy. It was originally developed out of the UK Met office and now Unidata partially helps to maintain the code base. It is currently the only viable option of plotting data on a map in Python. There are other options, but they are either no longer maintained or are not yet ready for "prime time".
- Cartopy: Projections, Geopolitical reference lines

https://scitools.org.uk/cartopy/docs/latest/index.html

Let's start by making some maps and plotting some map basics on them.

List of Common Projections:
- PlateCarree
- LambertConformal
- Mercator
- Robinson

Cartopy Projections: https://scitools.org.uk/cartopy/docs/latest/reference/crs.html#list-of-projections

### PlateCarree

To set a map projection we use a keyword argument on the `plt.subplot()` call called `projection`. In setting this projection we are telling our code what projection we want out output to be in.

```python
ax = plt.subplot(111, projection=ccrs.PlateCarree())
```

The projection from Cartopy is accessed using the alias (`ccrs`) we created when we imported that part of the module. Then each projection is a function where you can make some modifications to how the map is plotted. For example, with the `PlateCarree()` projection, you can move the central longitude to some value other than 0E, which is the default, by setting the `central_longitude` keyword.

PlateCarree Projection: https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#platecarree

In [ ]:
# Set a map coordinate reference
# Standard plot map for lat/lon data is PlateCarree
mapcrs = 

fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection=mapcrs)

# Simple method to add coastlines


ax.set_title('Plate Carree Projection')

plt.show()

### Lambert Conformal

Let's plot only the CONUS using a Lambert Conformal projection and add some rivers and lake features to our plot.

Lambert Conformal Projection
```python
ccrs.LambertConformal(central_longitude=-95, central_latitude=40, standard_parallels=[25, 60])
```



Setting the extent is accomplished with the following code:
```python
ax.set_extent([LLLon, LLLat, URLon, URLat], ccrs.PlateCarree())
```
The above extent is set by defining the lower-left longitude, latitude coordinate and an upper-right longitude latitude coordinate in degrees.

### Adding Map Features

The cfeature portion of the Cartopy library is a convenient tool to add some of the most common geo-political lines to provide some geographic reference to the data being plotted. These are shapefiles that are provided by Natural Earth Data (https://www.naturalearthdata.com) and you can access different scales (1:100 million, 1:50 million, and 1:10 million). For example, to get the coastlines for the scale 1:50 million you would use:

```python
cfeature.COASTLINE.with_scale('50m')
```

This is then paired with the axis method `add_feature` to then plot the shapefile on the particular axes, where you can modify aspects of the shapefiles like the color, facecolor, etc.). For example, adding Rivers with the 1:10 million scale and coloring them blue:

```python
ax.add_feature(cfeature.RIVERS.with_scale('10m'), color='tab:blue')
```

Here is the list of Natural Earth Features that have easy access methods via Cartopy (note the capitalization is important here):
* COASTLINE
* BORDERS
* STATES (US and Canada and just a few others)
* RIVERS
* LAKES
* OCEAN
* LAND

https://scitools.org.uk/cartopy/docs/latest/reference/feature.html

In [ ]:
# Create the same map as above, but for a Lambert Conformal Projection
# Centered on -100E Longitude, 40N latitude

mapcrs = 

fig = plt.figure(figsize=(15, 12))
ax = fig.add_subplot(111, projection=mapcrs)

ax.set_extent([-130, -65, 20, 55], ccrs.PlateCarree())

ax.add_feature(cfeature.COASTLINE.with_scale('50m'))


plt.title('Lambert Conformal Projection')
plt.show()

## Read Gridded Data and Plot on a Map

Now let's plot some data on our maps!

Let's bring in some atmospheric pressure level data for air temperature (and eventually geopotential heights) from NCEP/NCAR Reanalysis data. These files are available in a manner which makes them accessible remotely, meaning we can put in the link to the data and not have to have he files locally. This can happen because of the OPeNDAP protocal available through THREDDS, which hosts many Earth System Science datasets.

Here is a link to a THREDDS server hosted by NOAA that contains a number of different types of datasets.

https://psl.noaa.gov/thredds/catalog/Datasets/catalog.html

Navigate to the `ncep.reanalysis` folder, then to the `pressure` folder and select the `air.2021.nc` file. There are four different methods to access this data file and we want to use OPENDAP. Click that link and you'll be brought to a page that describes what is contained in the data file. What we are most interested in is the link at the top of the page in the box labeled Data URL. Copy that link to be able to use Xarray to read this data file remotely. 

In [ ]:
ds_air = xr.open_dataset('')

Let's take a look at the data file now that we have gained access to it. Note that we have not actually read all of the data from this file, only enough (e.g., the metadata) that we can know what data is in the file and some basic information about the file. Not until we do something (like plotting or doing a calculation) will the data actually be downloaded.

Wow, that is a lot of times available. We definitely don't need all of those. Let's subset via the time variable using the Xarray `.sel` method. Additionally, let's also select for just the 850 hPa pressure level. We can do both selections at the same time.

In [ ]:
date = datetime(2021, 3, 8, 12)

ds_air = ds_air.sel()

Note that the name of our data variable in the file is called `air`, so let's take a look at that data after our successful subsetting.

## Color-filled Contours
Alright, so now we have some data (that spans the whole globe) and now we want to plot it on a map using Cartopy!

Matplotlib color-filled contours work in the exact same manner as the contours except the name of the color-filled contours function is `contourf`, short for contour-filled.

At plot time we will convert the air temperatures from Kelvin (the unit of the data in the file) to Celisus by subtracting 273.15. Additionally, since we are now colorfilling our plot we need a method to place a colorbar on our figure so anyone looking at our figure would be able to determine what the values of our color-fill represent.

```python
cf = ax.contourf(ds_air.lon, ds_air.lat, ds_air.air-273.15)

plt.colorbar(cf, ax=ax, orientation='horizontal', pad=0, aspect=50)
```
Some key things in the above code. We first give a variable name to our color-filled contours, this is similar to what we did with contours as we need information from the plotting for making our colorbar (or adding labels for our regular contours). There is nothing special about what we use for the name, however, commonly `cs` is used for regular contours and `cf` is used for color-filled contours.


In [ ]:
plt.figure(1, figsize=(15, 15))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

cf = ax.contourf()

ax.coastlines()

plt.show()

### Shifting Our Plot
Why is there a white line in the middle of our plot?

Our data are organized from 0E Longitude to 357.5 Longitude, so we can't contour across that boundary natively. Cartopy does offer a function to add a cyclic point, but that goes beyond the scope of what we wish to do today. Instead we can shift our map to plot from 0E to 360E instead of -180E to 180E, which is the Cartopy default. This can be accomplished by changing the projection of the access and adding a `transform` keyword argument to our filled-contours to plot on the altered projection.

## Color-filled and Regular Contours

Now let's combine our air temperature with some geopotential height contours. To do so, we need to bring in the height data, which is in a different file from the same source as our air temperature data.

In [ ]:
ds_hght = xr.open_dataset('https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis/pressure/hgt.2021.nc')
ds_hght = ds_hght.sel(time=date, level=850)

Now just like having multiple lines on a line plot we simple add more plots to the axes we wish to plot on. Let's keep our color-filled contours for air temperature and add black contours of geopotential height with the following levels: `np.arange(0, 100000, 30)`.

In [ ]:
plt.figure(2, figsize=(20, 20))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

cf = ax.contourf(ds_air.lon, ds_air.lat, ds_air.air-273.15,
                np.arange(-50, 50, 1), cmap=plt.cm.coolwarm)
plt.colorbar(cf, ax=ax, orientation='horizontal', pad=0, aspect=50)

# Add Contours

ax.coastlines(color='grey')

plt.show()

## Change Projections

Even though we have global data, maybe we only want to plot the data from over the CONUS and plot is on a differen projection.

Let's work to subset our data to only be from 15 to 60N and from -140 to -55E for both the air temperature and geopotential heights.

Note: When performing the slicing of our lat and lon variables, it is important to know how they are structured (e.g., are the higher or lower values first) and what the range of values are (e.g., are longitudes in -180 to 180 or 0 to 360).

In [ ]:
ds_air_US = ds_air.sel(lat=slice(), lon=slice())
ds_hght_US = ds_hght.sel()

Now that we have subset our data, let's plot on a Lambert Conformal projection with a central_longitude of -100 and a central_latitude of 40.

Begin by seting up the new map projection coordinate reference system, and don't forget to add the `transform` keywork argument to any data that you wish to plot. The data format remains `ccrs.PlateCarree()`.

In [ ]:
mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=40)

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection=mapcrs)

ax.set_extent([-124, -70, 20, 52], ccrs.PlateCarree())

# Add Colorfilled Contours of Air Temperature
cf = 

# Add Contours of Geopotential Height
cs = 

ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))

ax.set_title(f'850-hPa Temperature (\u00b0C), Geopotential Height (m)', loc='left')
ax.set_title(f'Valid: {date} UTC', loc='right')

plt.show()

## Exercise #1
Plot the global data on a different projection that is still appropriate for global data (e.g., a Robinson projection).

## Exercise #2
Choose a different day and time to plot from 2020 (you'll need to pull in a different data file).

Potential Data Files: https://psl.noaa.gov/thredds/catalog/Datasets/ncep.reanalysis/pressure/catalog.html